<a href="https://colab.research.google.com/github/MJMortensonWarwick/DSML2223/blob/main/1_1_Regression_(Stats_vs_ML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression in Scikit-Learn

### Traditional Statistics Approach
As above, its a Linear Regression tutorial (yay), using the popular (indeed some may say seminal) machine learning library scikit-learn. As per usual, we will start by installing and importing:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd  
import seaborn as sns 

# Only works on Jupyter/Anaconda
%matplotlib inline  

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

We will also use one of the inbuilt datasets included with _scikit\-learn_:

In [ ]:
# import a standard dataset - the Boston house price index
from sklearn.datasets import load_boston
boston_dataset = load_boston()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

As the comment says, this is a well known dataset ... indeed Andrew Ng's first dataset in his first lecture of the first mainstream MOOC in machine learning. Let's look at it:

In [ ]:
# show the dataset
print(boston_dataset)

# print a return space
print('\n')

# show the keys
print(boston_dataset.keys())

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]]), 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
       18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
       15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
       13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
       21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
       35.4, 24.7, 31.6, 23.3, 19.6, 1

As we can see this is actually a dictionary with 'data' (the X's we can use), 'target' (the Y's), 'feature_names' (the names of all of the columns that represent X), 'DESCR' (a description of the data) and 'filename' (I'll let you guess). With this in mind let's add the X's to a DataFrame:

In [ ]:
# convert to data frame using Pandas
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


We will also create a variable for the Y value:

In [ ]:
# create a separate Y value
boston_Y = boston_dataset.target
boston_Y

array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
       18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
       15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
       13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
       21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
       35.4, 24.7, 31.6, 23.3, 19.6, 18.7, 16. , 22.2, 25. , 33. , 23.5,
       19.4, 22. , 17.4, 20.9, 24.2, 21.7, 22.8, 23.4, 24.1, 21.4, 20. ,
       20.8, 21.2, 20.3, 28. , 23.9, 24.8, 22.9, 23.9, 26.6, 22.5, 22.2,
       23.6, 28.7, 22.6, 22. , 22.9, 25. , 20.6, 28.4, 21.4, 38.7, 43.8,
       33.2, 27.5, 26.5, 18.6, 19.3, 20.1, 19.5, 19.5, 20.4, 19.8, 19.4,
       21.7, 22.8, 18.8, 18.7, 18.5, 18.3, 21.2, 19.2, 20.4, 19.3, 22. ,
       20.3, 20.5, 17.3, 18.8, 21.4, 15.7, 16.2, 18. , 14.3, 19.2, 19.6,
       23. , 18.4, 15.6, 18.1, 17.4, 17.1, 13.3, 17.8, 14. , 14.4, 13.4,
       15.6, 11.8, 13.8, 15.6, 14.6, 17.8, 15.4, 21

From here we can build our model. Let's recall that the model we want is of the form:

$ Y = a + b_1x_1 + b_2x_2 + [...] + b_13x_13 + e$

Where 
$Y$ is our target variable (boston_Y), $a$ is the intercept, the various $b$ values (1 to 13) represent the effect of the corresponding $x$ values (1 to 13) on the slope and $e$ is the error.

We start by defining a model and 'fitting' it to the data:

In [ ]:
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression()

# fit the model to the training data
lin_model_fit = lin_model.fit(boston, boston_Y)

Here we have created a Linear Regression object (lin_model) and then an object based on fitting it to our data. In other words, lin_model_fit is the Linear Regression process applied to our data.

From this object we can establish the remainder of our formula:

In [ ]:
# print the alpha value of the model (intercept)
print("Alpha/intercept (a)")
print(lin_model_fit.intercept_)

print("\n")

# print the beta values of the model (co-efficients)
betas = lin_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients (b1 to b13)")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas[counter], 4)))
    counter +=1

Alpha/intercept (a)
36.459488385090125


Beta weights/co-efficients (b1 to b13)
-----------------------------------------
CRIM: -0.108
ZN: 0.0464
INDUS: 0.0206
CHAS: 2.6867
NOX: -17.7666
RM: 3.8099
AGE: 0.0007
DIS: -1.4756
RAD: 0.306
TAX: -0.0123
PTRATIO: -0.9527
B: 0.0093
LSTAT: -0.5248


We can also calculate the root mean squared error (RMSE) and the $R^2$ score:

In [ ]:
# predict every Y value in the dataset
boston_predict = lin_model_fit.predict(boston)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(boston_Y, boston_predict)))
r2 = r2_score(boston_Y, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print(f'RMSE is {rmse}')
print(f'R2 score is {r2}')

Model performance
--------------------------------------
RMSE is 4.679191295697281
R2 score is 0.7406426641094095


Overall this is a good model - 
 scores above 0.7 are considered very much publishable/successful when building regression models! RMSE, the square root of the sum of each error term squared, is more domain specific (and often used for module comparison), but in this case means our forecasted property prices are incorrect by, on average, \$4,679. Given that many properties sell for more than $50,000 this is not too bad.

Keep scrolling, we'll next relook at this with a more "machine learning" mindset :)

### Machine Learning Approach
We'll redo our linear regression and this time we'll make it more "machine-learning-y". Actually, arguably, all we need to do is via splitting our data into training and test:

In [ ]:
# split data into training and test
from sklearn.model_selection  import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(boston, boston_Y, test_size = 0.2)

# print the shapes to check everything is OK
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(404, 13)
(102, 13)
(404,)
(102,)


The "shapes" shown here are number of rows by number of columns ... i.e. the shape of "X_train" is 404 rows by 13 columns. We'd expect the 13 columns (13 X's), and we can quickly do some math on the 506 row numbers ... 
 ... to see that 404 is rougly 80 % of the data (our "test_size" was 20% so our training data should be 80%) and that this all looks correct. The Y "shapes" show only rows but this is because we have only 1x value (1x column).

Using these subsets we can train the Linear Regression model on "X_train" and "Y_train", and then use this model to predict "X_test". If we can compare the predictions on "X_test" with the real values of "Y_test" this gives us a measure of how well our computer has learned to predict house prices:

In [ ]:
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression()

# fit the model to the training data
lin_model_fit = lin_model.fit(X_train, Y_train)

# predict the data
boston_predict = lin_model_fit.predict(X_test)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(Y_test, boston_predict)))
r2 = r2_score(Y_test, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

Model performance
--------------------------------------
RMSE is 4.313037645012264
R2 score is 0.7566309647954168




Compared to the previous Notebook our RMSE is lower and 
 higher (i.e. better in both cases)! This is a little suprising but probably just chance on how the data was split (random error means the metrics are superior). However, we can conclude that this hasn't made our model worse.

What it has done, however, is meant we have a model that is tested against its ability to make predictions on unseen data ... i.e. that it has learned about the process of predicting house prices and can do this well.

#### L1 Regularisation (LASSO)
$L1$ Regression is performed in much the same way as Linear Regression. We have one other value (a hyperparameter - more on these later) of $\alpha$ which determines how much influence the 
 penalty has on how the line is fit in the model. We will dodge the issue of what this should be for now and just randomly set it as 0.5

In [ ]:
from sklearn.linear_model import Lasso
l1_model = Lasso(alpha=0.5)

# fit the model to the training data
l1_model_fit = l1_model.fit(X_train, Y_train)

# predict the data
boston_predict = l1_model_fit.predict(X_test)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(Y_test, boston_predict)))
r2 = r2_score(Y_test, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

Model performance
--------------------------------------
RMSE is 4.517395047468784
R2 score is 0.7330223132991698




Performance is slightly lower than for our normal linear model. However... this is somewhat to be expected and actually performance is only slightly worse. This may be a small price to pay for ensuring that our model is a little more robust to changing data.

Where we should see a much more fundamental difference is in the beta-weights/coefficients of the two models (the $b$ values). Let's compare the two:

In [ ]:
# print the beta values of the model (co-efficients)
betas = lin_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - UNREGULARISED")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l1 = l1_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - LASSO")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l1[counter], 4)))
    counter +=1

Beta weights/co-efficients - UNREGULARISED
-----------------------------------------
CRIM: -0.1049
ZN: 0.0571
INDUS: 0.0576
CHAS: 2.7284
NOX: -20.8549
RM: 3.3444
AGE: 0.0104
DIS: -1.5142
RAD: 0.3726
TAX: -0.0131
PTRATIO: -1.0377
B: 0.0093
LSTAT: -0.5826


Beta weights/co-efficients - LASSO
-----------------------------------------
CRIM: -0.0799
ZN: 0.0627
INDUS: -0.0
CHAS: 0.0
NOX: -0.0
RM: 2.0658
AGE: 0.0122
DIS: -0.9604
RAD: 0.3238
TAX: -0.0157
PTRATIO: -0.8232
B: 0.0099
LSTAT: -0.7091


As we can see some of the beta weights are now 0 ("INDUS", "CHAS" and "NOX" - of which "CHAS" and "NOX" were fairly large weights in the original model). We have effectively removed these features (X's) from the model - they are now (for instance) $0 * INDUS$ which will obviously be zero and therefore won't change the calculation of $Y$. We have performed feature selection - removing X's from the model.

We have also minimised the impact (weight) of all the X's. For instance, "RM" had a beta of 3.77 in the original model and only 0.74 in the LASSO model.

Overall we have made the model less reliant on certain features and removed others entirely (reducing model complexity). Both these elements mean our model is more robust/protected against over-fitting ... and the cost is only 0.05 in terms of $R^2$ score.

#### L2 Regularisation (Ridge)
$L2$ regularisation is performed in a very similar way in scikit-learn ... and again we will set the hyperparameter $\alpha$ to the arbitrary value of 0.5:

In [ ]:
from sklearn.linear_model import Ridge
l2_model = Ridge(alpha=0.75)

# fit the model to the training data
l2_model_fit = l2_model.fit(X_train, Y_train)

# predict the data
boston_predict = l2_model_fit.predict(X_test)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(Y_test, boston_predict)))
r2 = r2_score(Y_test, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

Model performance
--------------------------------------
RMSE is 4.263553602085966
R2 score is 0.7621833387754423




Here we have model performance that slightly improves upon the original model ... while also adding regularisation protection. Let's compare the three beta weights:

In [ ]:
# print the beta values of the model (co-efficients)
betas = lin_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - UNREGULARISED")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l1 = l1_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - LASSO")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l1[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l2 = l2_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - RIDGE")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l2[counter], 4)))
    counter +=1

Beta weights/co-efficients - UNREGULARISED
-----------------------------------------
CRIM: -0.1049
ZN: 0.0571
INDUS: 0.0576
CHAS: 2.7284
NOX: -20.8549
RM: 3.3444
AGE: 0.0104
DIS: -1.5142
RAD: 0.3726
TAX: -0.0131
PTRATIO: -1.0377
B: 0.0093
LSTAT: -0.5826


Beta weights/co-efficients - LASSO
-----------------------------------------
CRIM: -0.0799
ZN: 0.0627
INDUS: -0.0
CHAS: 0.0
NOX: -0.0
RM: 2.0658
AGE: 0.0122
DIS: -0.9604
RAD: 0.3238
TAX: -0.0157
PTRATIO: -0.8232
B: 0.0099
LSTAT: -0.7091


Beta weights/co-efficients - RIDGE
-----------------------------------------
CRIM: -0.1028
ZN: 0.0581
INDUS: 0.0343
CHAS: 2.6228
NOX: -14.8011
RM: 3.378
AGE: 0.0051
DIS: -1.4251
RAD: 0.3587
TAX: -0.0136
PTRATIO: -0.978
B: 0.0097
LSTAT: -0.5878


As with L1/LASSO regression all our beta values are lower than in the original model - and therefore each feature is less influential. However, none have been reduced to zero. L2 cannot perform feature selection (deleting X's) unlike with L1.

So better performance and regularisation added - seems like a good deal right? However, before we call it a day we have one other regularisation method we may try.

#### ElasticNet
ElasticNet uses both $L1$ and $L2$ regularisation. We now have an additonal value to set (hyperparameter - still going to come back to these) the l1_ratio. As the name suggests the l1_ratio determines the proportion that is L1 versus L2 so "1" would be just L1 and "0" would just be L2. We'll arbitrarily go with 0.5 again.

In [ ]:
from sklearn.linear_model import ElasticNet
enet_model = ElasticNet(alpha=0.5, l1_ratio=0.25)

# fit the model to the training data
enet_model_fit = enet_model.fit(X_train, Y_train)

# predict the data
boston_predict = enet_model_fit.predict(X_test)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(Y_test, boston_predict)))
r2 = r2_score(Y_test, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

Model performance
--------------------------------------
RMSE is 4.672882408344411
R2 score is 0.7143274394848722




Unsurprisingly the result is somewhere between the results of L1 and L2. Let's also check the weights:

In [ ]:
# print the beta values of the model (co-efficients)
betas = lin_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - UNREGULARISED")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l1 = l1_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - LASSO")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l1[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l2 = l2_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - RIDGE")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l2[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_enet = enet_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - ELASTICNET")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_enet[counter], 4)))
    counter +=1

Beta weights/co-efficients - UNREGULARISED
-----------------------------------------
CRIM: -0.1049
ZN: 0.0571
INDUS: 0.0576
CHAS: 2.7284
NOX: -20.8549
RM: 3.3444
AGE: 0.0104
DIS: -1.5142
RAD: 0.3726
TAX: -0.0131
PTRATIO: -1.0377
B: 0.0093
LSTAT: -0.5826


Beta weights/co-efficients - LASSO
-----------------------------------------
CRIM: -0.0799
ZN: 0.0627
INDUS: -0.0
CHAS: 0.0
NOX: -0.0
RM: 2.0658
AGE: 0.0122
DIS: -0.9604
RAD: 0.3238
TAX: -0.0157
PTRATIO: -0.8232
B: 0.0099
LSTAT: -0.7091


Beta weights/co-efficients - RIDGE
-----------------------------------------
CRIM: -0.1028
ZN: 0.0581
INDUS: 0.0343
CHAS: 2.6228
NOX: -14.8011
RM: 3.378
AGE: 0.0051
DIS: -1.4251
RAD: 0.3587
TAX: -0.0136
PTRATIO: -0.978
B: 0.0097
LSTAT: -0.5878


Beta weights/co-efficients - ELASTICNET
-----------------------------------------
CRIM: -0.0884
ZN: 0.0681
INDUS: -0.003
CHAS: 0.1618
NOX: -0.0
RM: 1.4126
AGE: 0.0179
DIS: -0.9787
RAD: 0.3658
TAX: -0.0174
PTRATIO: -0.8684
B: 0.0095
LSTAT: -0.7538


Again, as we may expect, the results are somewhere between the two. "CHAS" and "NOX" have betas of zero (have been removed) but not "INDUS". All other beta weights are reduced to somewhere between the L1 and L2 models.

**BONUS! Which is best?**
Hahahahaha ... you know what I'm going to say:

## IT DEPENDS!
If you are not worried about overfitting use the original model (but if you don't care about overfitting are you doing ML?). If you have a lot of features and want some removed you may use L1 or Elastic Net. If you care most about performance use the L2 model. In this case I'd probably go with L2 as there aren't a lot of features and the performance is best.